In [1]:
!git clone https://github.com/Dzhambo/MBD.git

Cloning into 'MBD'...
remote: Enumerating objects: 390, done.
remote: Counting objects: 100% (390/390), done.
remote: Compressing objects: 100% (196/196), done.
remote: Total 390 (delta 232), reused 313 (delta 178), pack-reused 0 (from 0)
Receiving objects: 100% (390/390), 143.82 KiB | 6.25 MiB/s, done.
Resolving deltas: 100% (232/232), done.


**Install dependencies**

In [2]:
!pip install -r /kaggle/working/MBD/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.4/163.4 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.1/151.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.3/812.3 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 MB 6.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**Download Dataset**

In [3]:
from huggingface_hub import hf_hub_download 
 
hf_hub_download(repo_id="ai-lab/MBD-mini", filename="ptls.tar.gz", repo_type="dataset", local_dir="/kaggle/working/")
hf_hub_download(repo_id="ai-lab/MBD-mini", filename="targets.tar.gz", repo_type="dataset", local_dir="/kaggle/working/")

ptls.tar.gz:   0%|          | 0.00/1.40G [00:00<?, ?B/s]

targets.tar.gz:   0%|          | 0.00/7.93M [00:00<?, ?B/s]

'/kaggle/working/targets.tar.gz'

In [4]:
!tar -xf ptls.tar.gz
!tar -xf targets.tar.gz

In [5]:
import pandas as pd
import numpy as np
import os

import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import types as T

spark_conf = pyspark.SparkConf()
spark_conf.setMaster("local[*]").setAppName("JoinModality")
spark_conf.set("spark.driver.maxResultSize", "16g")
spark_conf.set("spark.executor.memory", "32g")
spark_conf.set("spark.executor.memoryOverhead", "16g")
spark_conf.set("spark.driver.memory", "32g")
spark_conf.set("spark.driver.memoryOverhead", "16g")
spark_conf.set("spark.cores.max", "24")
spark_conf.set("spark.sql.shuffle.partitions", "200")
spark_conf.set("spark.local.dir", "../../spark_local_dir")


spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
spark.sparkContext.getConf().getAll()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/01/16 23:54:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/16 23:54:40 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.app.startTime', '1737071680172'),
 ('spark.app.id', 'local-1737071681384'),
 ('spark.executor.memoryOverhead', '16g'),
 ('spark.driver.memory', '32g'),
 ('spark.driver.maxResultSize', '16g'),
 ('spark.app.name', 'JoinModalit

In [6]:
spark

In [7]:
TRX_DATA_PATH = '/kaggle/working/ptls/trx/'
GEO_DATA_PATH = '/kaggle/working/ptls/geo/'
DIAL_DATA_PATH = '/kaggle/working/ptls/dialog/'

MM_DATA_PATH = '/kaggle/working/MBD/scenario_mbd/data/mm_dataset/'

In [8]:
def rename_col(df, prefix, col_id='client_id'):
    new_column_names = [f"{prefix}_{col}" for col in df.columns if col != col_id]
    old_column_names = [col for col in df.columns if col != col_id]
    for old_col, new_col in zip(old_column_names, new_column_names):
        df = df.withColumnRenamed(old_col, new_col)
    return df

In [9]:
from tqdm.notebook import tqdm

for fold in tqdm(range(0, 5)):
    trx = spark.read.parquet(os.path.join(TRX_DATA_PATH, f'fold={fold}'))
    geo = spark.read.parquet(os.path.join(GEO_DATA_PATH, f'fold={fold}'))
    dial = spark.read.parquet(os.path.join(DIAL_DATA_PATH, f'fold={fold}'))
    
    trx = rename_col(trx, 'trx')
    geo = rename_col(geo, 'geo')
    dial = rename_col(dial, 'dial')
    
    mm_dataset = trx.join(geo, on='client_id', how='outer').join(dial, on='client_id', how='outer')
    mm_dataset.write.mode('overwrite').parquet(os.path.join(MM_DATA_PATH, f'fold={fold}'))

  0%|          | 0/5 [00:00<?, ?it/s]

In [10]:
# spark.read.parquet(os.path.join(MM_DATA_PATH, f'fold=1'))

**Targets**

In [11]:
from ptls.preprocessing import PysparkDataPreprocessor

In [12]:
TARGETS_DATA_PATH = '/kaggle/working/targets'

In [13]:
preprocessor_target = PysparkDataPreprocessor(
    col_id="client_id",
    col_event_time="mon",
    event_time_transformation="dt_to_timestamp",
    cols_identity=["target_1", "target_2", "target_3", "target_4"],
)

In [16]:
MMT_DATA_PATH = '/kaggle/working/MBD/scenario_mbd/data/mm_supervised/'

for fold in range(5):
    targets = spark.read.parquet(os.path.join(TARGETS_DATA_PATH , f'fold={fold}'))
    mm_dataset = spark.read.parquet(os.path.join(MM_DATA_PATH , f'fold={fold}'))
    
    targets = preprocessor_target.fit_transform(targets)
    mm_dataset = mm_dataset.join(targets, on='client_id', how='left').drop(*['trans_count', 'diff_trans_date'])
    mm_dataset.write.parquet(os.path.join(MMT_DATA_PATH, f'fold={fold}'))

In [16]:
spark.stop()

In [ ]:
!pip list

In [18]:
# %load /kaggle/working/MBD/scenario_mbd/conf/trx_coles.yaml
new_config = """fold: 3
defaults:
  - _self_
  - dataset_unsupervised/parquet_trx
  - inference/inference_trx
  - downstream/downstream

seed_everything: 42
logger_name: trx_coles_${fold}
model_path: scenario_mbd/models/${logger_name}.p
embed_file_name: ${logger_name}

data_module:
  _target_: ptls.frames.PtlsDataModule
  train_data:
    _target_: ptls.frames.coles.ColesIterableDataset
    splitter:
      _target_: ptls.frames.coles.split_strategy.SampleSlices
      split_count: 5
      cnt_min: 32
      cnt_max: 180
    data: ${dataset_unsupervised.train}
  valid_data:
    _target_: ptls.frames.coles.ColesIterableDataset
    splitter:
      _target_: ptls.frames.coles.split_strategy.SampleSlices
      split_count: 5
      cnt_min: 32
      cnt_max: 180
    data: ${dataset_unsupervised.valid}
  train_batch_size: 512
  train_num_workers: 16
  valid_batch_size: 128
  valid_num_workers: 16
  
trainer:
  precision: "bf16"
  accelerator: "gpu"
  devices: 1
  max_epochs: 15
  limit_val_batches: 5000
  gradient_clip_val: 0.5
  deterministic: true
  checkpoints_every_n_val_epochs: 1
  fast_dev_run: false
  
pl_module:
  _target_: ptls.frames.coles.CoLESModule
  validation_metric:
    _target_: ptls.frames.coles.metric.BatchRecallTopK
    K: 4
    metric: cosine
  seq_encoder:
    _target_: ptls.nn.RnnSeqEncoder
    trx_encoder:
      _target_: ptls.nn.TrxEncoder
      norm_embeddings: false
      embeddings_noise: 0.003
      embeddings:
        event_type:
          in: 58
          out: 24
        event_subtype:
          in: 59
          out: 24
        src_type11:
          in: 85
          out: 24
        src_type12:
          in: 349
          out: 24
        dst_type11:
          in: 84
          out: 24
        dst_type12:
          in: 417
          out: 12
        src_type22:
          in: 90
          out: 24
        src_type32:
          in: 91
          out: 24
      numeric_values:
        amount: log
    type: gru
    hidden_size: 256
  optimizer_partial:
    _partial_: true
    _target_: torch.optim.AdamW
    lr: 0.001
  lr_scheduler_partial:
    _partial_: true
    _target_: torch.optim.lr_scheduler.StepLR
    step_size: 3
    gamma: 0.9025"""

with open('/kaggle/working/MBD/scenario_mbd/conf/trx_coles.yaml', mode='w') as file:
    file.write(new_config)

In [19]:
# %load /kaggle/working/MBD/scenario_mbd/conf/dataset_unsupervised/parquet_trx.yaml
new_config = """train:
  _target_: ptls.data_load.datasets.ParquetDataset
  data_files: 
    - scenario_mbd/data/trx/fold=0
    - scenario_mbd/data/trx/fold=1
    - scenario_mbd/data/trx/fold=2
    - scenario_mbd/data/trx/fold=3
    - scenario_mbd/data/trx/fold=4
  i_filters:
    - _target_: ptls.data_load.iterable_processing.SeqLenFilter
      min_seq_len: 32
    - _target_: ptls.data_load.iterable_processing.iterable_seq_len_limit.ISeqLenLimit
      max_seq_len: 4096
    - _target_: ptls.data_load.iterable_processing.feature_filter.FeatureFilter
      drop_feature_names: 
        - client_id
        - target_1
        - target_2
        - target_3
        - target_4
    - _target_: ptls.data_load.iterable_processing.to_torch_tensor.ToTorch
  shuffle_files: true
valid:
  _target_: ptls.data_load.datasets.ParquetDataset
  data_files: 
    - scenario_mbd/data/trx/fold=${fold}
  i_filters:
    - _target_: ptls.data_load.iterable_processing.SeqLenFilter
      min_seq_len: 32
    - _target_: ptls.data_load.iterable_processing.iterable_seq_len_limit.ISeqLenLimit
      max_seq_len: 4096
    - _target_: ptls.data_load.iterable_processing.feature_filter.FeatureFilter
      drop_feature_names: 
        - client_id
        - target_1
        - target_2
        - target_3
        - target_4
    - _target_: ptls.data_load.iterable_processing.to_torch_tensor.ToTorch"""

with open('/kaggle/working/MBD/scenario_mbd/conf/dataset_unsupervised/parquet_trx.yaml', mode='w') as file:
    file.write(new_config)

In [20]:
# %load /kaggle/working/MBD/scenario_mbd/conf/inference/inference_trx.yaml
new_config = """num_workers: 16
batch_size: 256
gpus: 1
use_save_model: True
multimodal: False
dataset_train:
     _target_: ptls.data_load.datasets.ParquetDataset
     data_files: 
        - scenario_mbd/data/trx/fold=0
        - scenario_mbd/data/trx/fold=1
        - scenario_mbd/data/trx/fold=2
        - scenario_mbd/data/trx/fold=3
        - scenario_mbd/data/trx/fold=4
     i_filters:
      - _target_: ptls.data_load.iterable_processing.iterable_seq_len_limit.ISeqLenLimit
        max_seq_len: 4096
      - _target_: ptls.data_load.iterable_processing.feature_filter.FeatureFilter
        keep_feature_names: 
          - client_id
          - target_1
          - target_2
          - target_3
          - target_4
      - _target_: ptls.data_load.iterable_processing.to_torch_tensor.ToTorch
      - _target_: modules.processing.GetSplit
        start_month: 1 
        end_month: 12
        col_id: client_id
dataset_test:
    _target_: ptls.data_load.datasets.ParquetDataset
    data_files: 
        - scenario_mbd/data/trx/fold=${fold}
    i_filters:
        - _target_: ptls.data_load.iterable_processing.iterable_seq_len_limit.ISeqLenLimit
          max_seq_len: 4096
        - _target_: ptls.data_load.iterable_processing.feature_filter.FeatureFilter
          keep_feature_names: 
            - client_id
            - target_1
            - target_2
            - target_3
            - target_4
        - _target_: ptls.data_load.iterable_processing.to_torch_tensor.ToTorch
        - _target_: modules.processing.GetSplit
          start_month: 1 
          end_month: 12
          col_id: client_id
output:
  path: scenario_mbd/data/${embed_file_name}
  format: parquet

col_id: client_id
target_col_names:
  - target_1
  - target_2
  - target_3
  - target_4"""


with open('/kaggle/working/MBD/scenario_mbd/conf/inference/inference_trx.yaml', mode='w') as file:
    file.write(new_config)

In [21]:
!ls /kaggle/working/MBD/scenario_mbd/data

mm_dataset  mmt_dataset


In [22]:
!mv /kaggle/working/ptls/trx /kaggle/working/MBD/scenario_mbd/data
!mv /kaggle/working/ptls/geo /kaggle/working/MBD/scenario_mbd/data
!mv /kaggle/working/ptls/dialog /kaggle/working/MBD/scenario_mbd/data
# !mv /kaggle/working/MBD/scenario_mbd/data/trx /kaggle/working/MBD/scenario_mbd/data/mm_dataset 

In [ ]:
# !export FOLD=3
%cd /kaggle/working/MBD
!python ptls_train_inf_down_module.py \
        fold=3 \
        "dataset_unsupervised.train.data_files.3='/kaggle/working/MBD/scenario_mbd/data/trx/fold=4'"\
        "inference.dataset_train.data_files.3='/kaggle/working/MBD/scenario_mbd/data/trx/fold=4'" \
            --config-dir scenario_mbd/conf --config-name trx_coles

In [60]:
!mv /kaggle/working/MBD/scenario_mbd/data/mmt_dataset /kaggle/working/MBD/scenario_mbd/data/mm_supervised

**trx agg**

In [44]:
# %load /kaggle/working/MBD/scenario_mbd/conf/trx_agg.yaml
"""fold: 4
defaults:
  - _self_
  - dataset_unsupervised/parquet_trx
  - inference/inference_agg_trx
  - downstream/downstream


seed_everything: 42
logger_name: trx_agg_${fold}
model_path: scenario_mbd/models/${logger_name}.p
embed_file_name: ${logger_name}

pl_module:
    seq_encoder:
    _target_: ptls.nn.AggFeatureSeqEncoder
    embeddings:
        event_type:
          in: 58
          out: 24
        event_subtype:
          in: 59
          out: 24
        src_type11:
          in: 85
          out: 24
        src_type12:
          in: 349
          out: 24
        dst_type11:
          in: 84
          out: 24
        dst_type12:
          in: 417
          out: 12
        src_type22:
          in: 90
          out: 24
        src_type32:
          in: 91
          out: 24
    numeric_values:
      amount: log
    was_logified: false
    log_scale_factor: 1"""

In [45]:
new_config = """num_workers: 0
batch_size: 32
gpus: 1
use_save_model: True
multimodal: False
dataset_train:
     _target_: ptls.data_load.datasets.ParquetDataset
     data_files: 
          - trx_train_prepr.parquet
     i_filters:
     - _target_: ptls.data_load.iterable_processing.iterable_seq_len_limit.ISeqLenLimit
       max_seq_len: 128
     - _target_: ptls.data_load.iterable_processing.feature_filter.FeatureFilter
       keep_feature_names: 
        - client_id
        - target_1
        - target_2
        - target_3
        - target_4
     - _target_: ptls.data_load.iterable_processing.to_torch_tensor.ToTorch
     - _target_: modules.processing.GetSplit
       start_month: 1 
       end_month: 12
       col_id: client_id
dataset_test:
    _target_: ptls.data_load.datasets.ParquetDataset
    data_files: 
          - trx_train_prepr.parquet
    i_filters:
        - _target_: ptls.data_load.iterable_processing.iterable_seq_len_limit.ISeqLenLimit
          max_seq_len: 128
        - _target_: ptls.data_load.iterable_processing.feature_filter.FeatureFilter
          keep_feature_names: 
            - client_id
            - target_1
            - target_2
            - target_3
            - target_4
        - _target_: ptls.data_load.iterable_processing.to_torch_tensor.ToTorch
output:
  path: ${embed_file_name}
  format: parquet"""
with open('/kaggle/working/MBD/scenario_mbd/conf/inference/inference_agg_trx.yaml', mode='w') as file:
    file.write(new_config)

In [62]:
!python ptls_train_inf_down_module.py \
    fold=3 \
    "dataset_unsupervised.train.data_files.2='/kaggle/working/MBD/scenario_mbd/data/mm_dataset/fold=1'"\
    "inference.dataset_train.data_files.2='/kaggle/working/MBD/scenario_mbd/data/mm_dataset/fold=1'" \
      --config-dir scenario_mbd/conf --config-name trx_agg


Error merging override inference.dataset_train.data_files.2='/kaggle/working/MBD/scenario_mbd/data/mm_dataset/fold=1'
list index out of range
    full_key: inference.dataset_train.data_files[2]
    object_type=list

Set the environment variable HYDRA_FULL_ERROR=1 for a complete stack trace.


**Mutimodal**

In [20]:
# %load /kaggle/working/MBD/scenario_mbd/conf/dataset_unsupervised/multimodal_parquet_supervised.yaml
new_config = """train:
  _target_: ptls.data_load.datasets.ParquetDataset
  data_files:
    - scenario_mbd/data/mm_supervised/fold=0
    - scenario_mbd/data/mm_supervised/fold=1
    - scenario_mbd/data/mm_supervised/fold=2
    - scenario_mbd/data/mm_supervised/fold=3
    - scenario_mbd/data/mm_supervised/fold=4
  i_filters:
    - _target_: ptls.data_load.iterable_processing.SeqLenFilter
      min_seq_len: 32
    - _target_: ptls.data_load.iterable_processing.iterable_seq_len_limit.ISeqLenLimit
      max_seq_len: 4096
    - _target_: ptls.data_load.iterable_processing.to_torch_tensor.ToTorch
    - _target_: modules.processing.TargetToTorch
      col_target: target_1
    - _target_: modules.processing.GetSplit
      start_month: 1
      end_month: 12
      col_id: client_id
  shuffle_files: true
valid:
  _target_: ptls.data_load.datasets.ParquetDataset
  data_files:
    - scenario_mbd/data/mm_supervised/fold=${fold}
  i_filters:
    - _target_: ptls.data_load.iterable_processing.SeqLenFilter
      min_seq_len: 32
    - _target_: ptls.data_load.iterable_processing.iterable_seq_len_limit.ISeqLenLimit
      max_seq_len: 4096
    - _target_: ptls.data_load.iterable_processing.to_torch_tensor.ToTorch
    - _target_: modules.processing.TargetToTorch
      col_target: target_1
    - _target_: modules.processing.GetSplit
      start_month: 1
      end_month: 12
      col_id: client_id"""

with open('/kaggle/working/MBD/scenario_mbd/conf/dataset_unsupervised/multimodal_parquet_supervised.yaml', mode='w') as file:
    file.write(new_config)

In [18]:
# %load /kaggle/working/MBD/scenario_mbd/conf/matching_conf_trxdial.yaml
new_config = """fold: 4

defaults:
  - _self_
  - dataset_unsupervised/multimodal_parquet_supervised

logger_name: matching_trxdial_${fold}

inference_run: false
downstream_run: false

trainer:
    max_epochs: 50
    accelerator: gpu
    enable_progress_bar: True
    gradient_clip_val: 0.5
    log_every_n_steps: 50
    limit_val_batches: 512

data_module:
  _target_: ptls.frames.PtlsDataModule
  train_data:
    _target_: modules.matching.MultiModalDiffSplitIterableDataset
    splitters:
        trx:
          _target_: ptls.frames.coles.split_strategy.SampleSlices
          split_count: 2
          cnt_min: 32
          cnt_max: 180
        dial:
          _target_: ptls.frames.coles.split_strategy.SampleSlices
          split_count: 2
          cnt_min: 2
          cnt_max: 10
    data: ${dataset_unsupervised.train}
    source_features:
        trx:
          - trx_event_type
          - trx_event_subtype
          - trx_src_type11
          - trx_src_type12
          - trx_dst_type11
          - trx_dst_type12
          - trx_src_type22
          - trx_src_type32
          - trx_event_time
        dial:
          - dial_embedding
          - dial_event_time
    col_id: client_id
    col_time: event_time
    source_names:
        - trx
        - dial
  valid_data:
    _target_: modules.matching.MultiModalDiffSplitIterableDataset
    splitters:
        trx:
          _target_: ptls.frames.coles.split_strategy.NoSplit
        dial:
          _target_: ptls.frames.coles.split_strategy.NoSplit
    data: ${dataset_unsupervised.valid}
    source_features:
        trx:
          - trx_event_type
          - trx_event_subtype
          - trx_src_type11
          - trx_src_type12
          - trx_dst_type11
          - trx_dst_type12
          - trx_src_type22
          - trx_src_type32
          - trx_event_time
        dial:
          - dial_embedding
          - dial_event_time
    col_id: user_id
    col_time: event_time
    source_names:
        - trx
        - dial
  train_batch_size: 256
  train_num_workers: 255
  valid_batch_size: 256
  valid_num_workers: 255

pl_module:
  _target_: modules.matching.M3CoLESModule
  validation_metric:
    _target_: ptls.frames.coles.metric.BatchRecallTopK
    K: 1
    metric: cosine
  head:
    _target_: ptls.nn.Head
    input_size: 128
    use_norm_encoder: True
    hidden_layers_sizes: 
      - 128
      - 128
    objective: "regression"
    num_classes: 128
  seq_encoders:
      trx:
        _target_: ptls.nn.RnnSeqEncoder
        trx_encoder:
          _target_: ptls.nn.TrxEncoder
          norm_embeddings: false
          embeddings_noise: 0.003
          linear_projection_size: 32
          embeddings:
            event_type:
              in: 58
              out: 24
            event_subtype:
              in: 59
              out: 24
            src_type11:
              in: 85
              out: 24
            src_type12:
              in: 349
              out: 24
            dst_type11:
              in: 84
              out: 24
            dst_type12:
              in: 417
              out: 12
            src_type22:
              in: 90
              out: 24
            src_type32:
              in: 91
              out: 24
          numeric_values:
            amount: log
        type: gru
        hidden_size: 128
      dial: 
        _target_: ptls.nn.RnnSeqEncoder
        trx_encoder:
          _target_: ptls.nn.TrxEncoder
          embeddings_noise: 0.003
          linear_projection_size: 32
          custom_embeddings:
            embedding: 
              _target_: ptls.nn.trx_encoder.encoders.IdentityEncoder
              output_size: 768
        type: gru
        hidden_size: 128
  loss:
    _target_: ptls.frames.coles.losses.SoftmaxLoss
  optimizer_partial:
    _partial_: true
    _target_: torch.optim.AdamW
    lr: 0.001
    weight_decay: 1e-4
  lr_scheduler_partial:
    _partial_: true
    _target_: torch.optim.lr_scheduler.StepLR
    step_size: 1
    gamma: 0.9"""

with open('/kaggle/working/MBD/scenario_mbd/conf/matching_conf_trxdial.yaml', mode='w') as file:
    file.write(new_config)

In [37]:
spark.read.parquet(os.path.join(MMT_DATA_PATH, f'fold={1}'))

DataFrame[client_id: string, trx_event_time: array<bigint>, trx_amount: array<float>, trx_event_type: array<int>, trx_event_subtype: array<int>, trx_currency: array<int>, trx_src_type11: array<int>, trx_src_type12: array<int>, trx_dst_type11: array<int>, trx_dst_type12: array<int>, trx_src_type21: array<int>, trx_src_type22: array<int>, trx_src_type31: array<int>, trx_src_type32: array<int>, geo_event_time: array<bigint>, geo_geohash_4: array<int>, geo_geohash_5: array<int>, geo_geohash_6: array<int>, dial_event_time: array<bigint>, dial_embedding: array<array<float>>, event_time: array<bigint>, target_1: array<int>, target_2: array<int>, target_3: array<int>, target_4: array<int>]

In [36]:
spark.read.parquet(os.path.join(MMT_DATA_PATH, f'fold={1}')).select('event_time').show(2, truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------+
|event_time                                                                                                                                      |
+------------------------------------------------------------------------------------------------------------------------------------------------+
|[1646006400, 1648684800, 1651276800, 1653955200, 1656547200, 1659225600, 1661904000, 1664496000, 1667174400, 1669766400, 1672444800, 1675123200]|
|[1646006400, 1648684800, 1651276800, 1653955200, 1656547200, 1659225600, 1661904000, 1664496000, 1667174400, 1669766400, 1672444800, 1675123200]|
+------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 2 rows



In [48]:
spark.read.parquet(os.path.join(MMT_DATA_PATH, f'fold={1}')).select('trx_event_time').show(1, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [21]:
%cd /kaggle/working/MBD
!python ptls_train_inf_down_module.py \
    fold=3 \
    "dataset_unsupervised.train.data_files.3='scenario_mbd/data/mm_dataset_fold/fold=4'"\
        --config-dir scenario_mbd/conf --config-name matching_conf_trxdial

/kaggle/working/MBD
[2025-01-17 00:03:16,314][__main__][INFO] - Start train...
[2025-01-17 00:03:16,493][pytorch_lightning.utilities.rank_zero][INFO] - GPU available: True (cuda), used: True
[2025-01-17 00:03:16,495][pytorch_lightning.utilities.rank_zero][INFO] - TPU available: False, using: 0 TPU cores
[2025-01-17 00:03:16,495][pytorch_lightning.utilities.rank_zero][INFO] - HPU available: False, using: 0 HPUs
[2025-01-17 00:03:16,562][pytorch_lightning.loggers.tensorboard][WARNING] - Missing logger folder: lightning_logs/matching_trxdial_3
[2025-01-17 00:03:17,318][pytorch_lightning.accelerators.cuda][INFO] - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[2025-01-17 00:03:17,552][pytorch_lightning.callbacks.model_summary][INFO] - 
  | Name               | Type            | Params | Mode 
---------------------------------------------------------------
0 | _loss              | SoftmaxLoss     | 0      | train
1 | _seq_encoder       | RnnSeqEncoder   | 92.8 K | train
2 | _validation_metric |

**В 19 строке был пробел после цифры**

**В целом что-то крайне странное, гидра просто не может собрать нормально конфиг. Как будто проще написать mm + agg с нуля**

In [66]:
# %load scenario_mbd/conf/matching_conf_trxdial.yaml
new_config = """fold: 3

defaults:
  - _self_
  - dataset_unsupervised/multimodal_parquet_supervised

logger_name: matching_trxdial_${fold}

inference_run: false
downstream_run: false

trainer:
    max_epochs: 50
    accelerator: gpu
    enable_progress_bar: True
    gradient_clip_val: 0.5
    log_every_n_steps: 50
    limit_val_batches: 512

data_module:
  _target_: ptls.frames.PtlsDataModule
  train_data:
    _target_: modules.matching.MultiModalDiffSplitIterableDataset
    splitters:
        trx:
          _target_: ptls.frames.coles.split_strategy.SampleSlices
          split_count: 2
          cnt_min: 32
          cnt_max: 180
        dial:
          _target_: ptls.frames.coles.split_strategy.SampleSlices
          split_count: 2
          cnt_min: 2
          cnt_max: 10
    data: ${dataset_unsupervised.train}
    source_features:
        trx:
          - event_type
          - event_subtype
          - src_type11
          - src_type12
          - dst_type11
          - dst_type12
          - src_type22
          - src_type32
          - event_time
        dial:
          - embedding
          - event_time
    col_id: client_id
    col_time: event_time
    source_names:
        - trx
        - dial
  valid_data:
    _target_: modules.matching.MultiModalDiffSplitIterableDataset
    splitters:
        trx:
          _target_: ptls.frames.coles.split_strategy.NoSplit
        dial:
          _target_: ptls.frames.coles.split_strategy.NoSplit
    data: ${dataset_unsupervised.valid}
    source_features:
        trx:
          - event_type
          - event_subtype
          - src_type11
          - src_type12
          - dst_type11
          - dst_type12
          - src_type22
          - src_type32
          - event_time
        dial:
          - embedding
          - event_time
    col_id: user_id
    col_time: event_time
    source_names:
        - trx
        - dial
  train_batch_size: 256
  train_num_workers: 255
  valid_batch_size: 256
  valid_num_workers: 255

pl_module:
  _target_: modules.matching.M3CoLESModule
  validation_metric:
    _target_: ptls.frames.coles.metric.BatchRecallTopK
    K: 1
    metric: cosine
  head:
    _target_: ptls.nn.Head
    input_size: 128
    use_norm_encoder: True
    hidden_layers_sizes: 
      - 128
      - 128
    objective: "regression"
    num_classes: 128
  seq_encoders:
      trx:
        _target_: ptls.nn.RnnSeqEncoder
        trx_encoder:
          _target_: ptls.nn.TrxEncoder
          norm_embeddings: false
          embeddings_noise: 0.003
          linear_projection_size: 32
          embeddings:
            event_type:
              in: 58
              out: 24
            event_subtype:
              in: 59
              out: 24
            src_type11:
              in: 85
              out: 24
            src_type12:
              in: 349
              out: 24
            dst_type11:
              in: 84
              out: 24
            dst_type12:
              in: 417
              out: 12
            src_type22:
              in: 90
              out: 24
            src_type32:
              in: 91
              out: 24
          numeric_values:
            amount: log
        type: gru
        hidden_size: 128
      dial: 
        _target_: ptls.nn.RnnSeqEncoder
        trx_encoder:
          _target_: ptls.nn.TrxEncoder
          embeddings_noise: 0.003
          linear_projection_size: 32
          custom_embeddings:
            embedding: 
              _target_: ptls.nn.trx_encoder.encoders.IdentityEncoder
              size: 768
        type: gru
        hidden_size: 128
  loss:
    _target_: ptls.frames.coles.losses.SoftmaxLoss
  optimizer_partial:
    _partial_: true
    _target_: torch.optim.AdamW
    lr: 0.001
    weight_decay: 1e-4
  lr_scheduler_partial:
    _partial_: true
    _target_: torch.optim.lr_scheduler.StepLR
    step_size: 1
    gamma: 0.9"""
with open('/kaggle/working/MBD/scenario_mbd/conf/matching_conf_trxdial.yaml', mode='w') as file:
    file.write(new_config)

In [67]:
!python ptls_train_inf_down_module.py \
        fold=3 \
        "dataset_unsupervised.train.data_files.3='/kaggle/working/MBD/scenario_mbd/data/mm_dataset/fold=4'"\
            --config-dir scenario_mbd/conf --config-name matching_conf_trxdial

Could not override 'dataset_unsupervised.train.data_files.3'.
To append to your config use +dataset_unsupervised.train.data_files.3='/kaggle/working/MBD/scenario_mbd/data/mm_dataset/fold=4'
Key 'train' is not in struct
    full_key: dataset_unsupervised.train
    object_type=dict

Set the environment variable HYDRA_FULL_ERROR=1 for a complete stack trace.
